In [ ]:
# test cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeat
import pandas as pd
import geopandas as gpd
import xarray as xr
# import salem
import numpy as np
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.image import imread
from cartopy.io.shapereader import Reader
import matplotlib as mpl
from cartopy.feature import ShapelyFeature
import shapely.vectorized
import shapefile as shp  # Requires the pyshp package
import cmocean
# %matplotlib inline

mpl.rcParams['text.usetex'] = True
# Set font properties globally
plt.rcParams['font.family'] = "Times New Roman"
# Set font properties globally
plt.rcParams['font.size'] = 18.5
plt.rcParams['axes.unicode_minus'] = False
plt.figure(dpi=600)

plt.tight_layout()
fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111,projection=ccrs.PlateCarree())
# ax = plt.axes(projection=ccrs.Mercator())
ax.set_extent([-76,-80,43,44.5],crs=ccrs.PlateCarree())
ax.set_xticks(np.arange(-80,-76+1,1.0))
ax.set_yticks(np.arange(43,44.5+0.5,0.5))

# not showing NSEW
cardinal_labels = {"east": "", "west": "", "north": "", "south": ""}
lon_formatter = LongitudeFormatter(zero_direction_label=True, cardinal_labels=cardinal_labels)
lat_formatter = LatitudeFormatter(cardinal_labels=cardinal_labels)
ax.xaxis.set_major_formatter(lon_formatter)
ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
ax.yaxis.set_major_formatter(lat_formatter)
ax.yaxis.set_minor_locator(plt.MultipleLocator(1))

ax.coastlines()
# ax.add_feature(cfeat.LAND.with_scale('10m'), zorder=1)
ax.add_feature(cfeat.LAKES.with_scale('10m'), zorder=10, edgecolor='plum', linewidth=1.8, facecolor='none', alpha=0.7)

In [10]:
# test a single contour with contourf
def plot_contours_vmag(time_step, lon2D, lat2D, vmag_data, ux, uy, save_name):
    
    plt.tight_layout()
    aspectRatio = 10/8
    # cmap = cmocean.cm.dense
    cmap = 'PuBuGn'

    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111,projection=proj)
    
    ax.set_extent([-76,-80,43,44.5],crs=ccrs.PlateCarree())
    ax.set_xticks(np.arange(-80,-76+1,1.0))
    ax.set_yticks(np.arange(43,44.5+0.5,0.5))
    # not showing NSEW
    cardinal_labels = {"east": "", "west": "", "north": "", "south": ""}
    lon_formatter = LongitudeFormatter(zero_direction_label=True, cardinal_labels=cardinal_labels)
    lat_formatter = LatitudeFormatter(cardinal_labels=cardinal_labels)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.yaxis.set_minor_locator(plt.MultipleLocator(1))
    
    ax.coastlines()
    ax.add_feature(cfeat.LAND.with_scale('10m'), zorder=1)
    ax.add_feature(cfeat.LAKES.with_scale('10m'), zorder=10, edgecolor='goldenrod', facecolor='none', linewidth=2, alpha=0.7)

    # fig, ax = plt.subplots(figsize = (15, 9))
    # tcf = ax.contourf(lon2D, lat2D, vmag[time_step], levels = np.arange(0, 11, 1), cmap="RdBu_r",extend='both')
    tcf = ax.contourf(lon2D,lat2D,vmag_data[time_step],levels = np.arange(0, 17, 1),cmap=cmap,extend='both')
    ax.quiver(lon2D, lat2D, ux[time_step], uy[time_step],units='width',pivot='tip',width=0.001,scale=500,
              headwidth=6, headlength=4, color='k')
    # divider = make_axes_locatable(ax)
    # cax = divider.new_vertical(size='22.5%', pad=1.0)
    # fig.colorbar(tcf, cax=cax, ax=ax, orientation='horizontal')
    cb = fig.colorbar(tcf, extend='both',orientation='horizontal', location='top',aspect=35)
    cb.set_label(r'$\sqrt{u_{10}^2+v_{10}^2}$\,{\rm (m/s)}')

    fig.savefig(save_name, dpi=225)

In [ ]:
input_file_no_ext_uu = './2099/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_UU_2099'+'12'
input_file_no_ext_vv = './2099/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_VV_2099'+'12'
input_file_uu = input_file_no_ext_uu+'_mod.nc'
input_file_vv = input_file_no_ext_vv+'_mod.nc'
nc_data_uu = xr.open_dataset(input_file_uu)
nc_data_vv = xr.open_dataset(input_file_vv)
vmag = ((nc_data_uu.UU.values)**2.0+(nc_data_vv.VV.values)**2.0)**0.5

plot_contours_vmag(nc_data_uu.lon.values, nc_data_uu.lat.values, vmag, nc_data_uu.UU.values, nc_data_vv.VV.values, 1, ' ')

In [ ]:
def plot_contours_vmag_anim(time_step, lon2D, lat2D, vmag_data, ux, uy, save_name, contour):
    
    plt.tight_layout()
    aspectRatio = 10/8
    # cmap = cmocean.cm.dense
    cmap = 'PuBuGn'

    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111,projection=proj)
    
    ax.set_extent([-76,-80,43,44.5],crs=ccrs.PlateCarree())
    ax.set_xticks(np.arange(-80,-76+1,1.0))
    ax.set_yticks(np.arange(43,44.5+0.5,0.5))
    # not showing NSEW
    cardinal_labels = {"east": "", "west": "", "north": "", "south": ""}
    lon_formatter = LongitudeFormatter(zero_direction_label=True, cardinal_labels=cardinal_labels)
    lat_formatter = LatitudeFormatter(cardinal_labels=cardinal_labels)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.yaxis.set_minor_locator(plt.MultipleLocator(1))
    
    ax.coastlines()
    ax.add_feature(cfeat.LAND.with_scale('10m'), zorder=1)
    ax.add_feature(cfeat.LAKES.with_scale('10m'), zorder=10, edgecolor='goldenrod', facecolor='none', linewidth=2, alpha=0.7)

    # fig, ax = plt.subplots(figsize = (15, 9))
    # tcf = ax.contourf(lon2D, lat2D, vmag[time_step], levels = np.arange(0, 11, 1), cmap="RdBu_r",extend='both')
    tcf = ax.contourf(lon2D,lat2D,vmag_data[time_step],levels = np.arange(0, 17, 1),cmap=cmap,extend='both')
    ax.quiver(lon2D, lat2D, ux[time_step], uy[time_step],units='width',pivot='tip',width=0.001,scale=500,
              headwidth=6, headlength=4, color='m')
    # divider = make_axes_locatable(ax)
    # cax = divider.new_vertical(size='22.5%', pad=1.0)
    # fig.colorbar(tcf, cax=cax, ax=ax, orientation='horizontal')
    cb = fig.colorbar(tcf, extend='both',orientation='horizontal', location='top',aspect=35)
    cb.set_label(r'$\sqrt{u_{10}^2+v_{10}^2}$\,{\rm (m/s)}')

In [ ]:
# the animation
import matplotlib.animation as animation
import os
os.system("rm ./2099/anim/*.png")

mpl.rcParams['text.usetex'] = True
# Set font properties globally
plt.rcParams['font.family'] = "Times New Roman"
# Set font properties globally
plt.rcParams['font.size'] = 20
plt.rcParams['axes.unicode_minus'] = False
plt.figure(dpi=600)

input_file_no_ext_uu = './2099/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_UU_2099'+'12'
input_file_no_ext_vv = './2099/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_VV_2099'+'12'
input_file_uu = input_file_no_ext_uu+'_mod.nc'
input_file_vv = input_file_no_ext_vv+'_mod.nc'
nc_data_uu = xr.open_dataset(input_file_uu)
nc_data_vv = xr.open_dataset(input_file_vv)
vmag = ((nc_data_uu.UU.values)**2.0+(nc_data_vv.VV.values)**2.0)**0.5

for frame in np.arange(len(nc_data_uu.time)):
    plot_contours_vmag(frame, nc_data_uu.lon.values, nc_data_uu.lat.values, vmag, nc_data_uu.UU.values, nc_data_vv.VV.values, './2099/anim/2099-12-'+str(frame)+'.png')

# fig = plt.figure(figsize=(10,6))
# proj = ccrs.PlateCarree()
# ax = fig.add_subplot(111,projection=proj)

# ani = animation.FuncAnimation(fig, plot_contours_vmag, len(nc_data_uu.time), fargs=[nc_data_uu.lon.values, nc_data_uu.lat.values, vmag, nc_data_uu.UU.values, nc_data_vv.VV.values, ' '], interval=6.0)
# ani = animation.FuncAnimation(fig, plot_contours_vmag, 20, fargs=[nc_data_uu.lon.values, nc_data_uu.lat.values, vmag, nc_data_uu.UU.values, nc_data_vv.VV.values, ' '], interval=12.0)
# ani.save('./2099_anim.gif', dpi = 500)
# ani.save('./2099_anim.mp4', dpi = 500)
# plt.show()

In [11]:
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np

gif_path = "./2099_anim.gif"

frames = np.stack([iio.imread(f"./2099/anim/2099-12-{x}.png") for x in range(len(nc_data_uu.time))], axis=0)

iio.imwrite(gif_path, frames)

In [ ]:
# 2059
import matplotlib.animation as animation
import os
os.system("rm -rf ./2059/anim/12/")
os.system("mkdir -p ./2059/anim/12/")

mpl.rcParams['text.usetex'] = True
# Set font properties globally
plt.rcParams['font.family'] = "Times New Roman"
# Set font properties globally
plt.rcParams['font.size'] = 20
plt.rcParams['axes.unicode_minus'] = False
plt.figure(dpi=225)

input_file_no_ext_uu = './2059/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_UU_2059'+'12'
input_file_no_ext_vv = './2059/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_VV_2059'+'12'
input_file_uu = input_file_no_ext_uu+'_mod.nc'
input_file_vv = input_file_no_ext_vv+'_mod.nc'
nc_data_uu = xr.open_dataset(input_file_uu)
nc_data_vv = xr.open_dataset(input_file_vv)
vmag = ((nc_data_uu.UU.values)**2.0+(nc_data_vv.VV.values)**2.0)**0.5

for frame in np.arange(len(nc_data_uu.time)):
    plot_contours_vmag(frame, nc_data_uu.lon.values, nc_data_uu.lat.values, vmag, nc_data_uu.UU.values, nc_data_vv.VV.values, './2059/anim/12/2059-12-'+str(frame)+'.png')

import imageio.v3 as iio

gif_path = "./2059-12_anim.gif"

frames = np.stack([iio.imread(f"./2059/anim/12/2059-12-{x}.png") for x in range(len(nc_data_uu.time))], axis=0)

iio.imwrite(gif_path, frames)

In [ ]:
# 2039
import matplotlib.animation as animation
import os
os.system("rm -rf ./2039/anim/2/")
os.system("mkdir -p ./2039/anim/2/")

mpl.rcParams['text.usetex'] = True
# Set font properties globally
plt.rcParams['font.family'] = "Times New Roman"
# Set font properties globally
plt.rcParams['font.size'] = 20
plt.rcParams['axes.unicode_minus'] = False
plt.figure(dpi=225)

input_file_no_ext_uu = './2039/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_UU_2039'+'02'
input_file_no_ext_vv = './2039/'+'NEMO_GL_MPI-ESM1-2-HR_SSP5-RCP8.5_324x200-cfl3_VV_2039'+'02'
input_file_uu = input_file_no_ext_uu+'_mod.nc'
input_file_vv = input_file_no_ext_vv+'_mod.nc'
nc_data_uu = xr.open_dataset(input_file_uu)
nc_data_vv = xr.open_dataset(input_file_vv)
vmag = ((nc_data_uu.UU.values)**2.0+(nc_data_vv.VV.values)**2.0)**0.5

for frame in np.arange(len(nc_data_uu.time)):
    plot_contours_vmag(frame, nc_data_uu.lon.values, nc_data_uu.lat.values, vmag, nc_data_uu.UU.values, nc_data_vv.VV.values, './2039/anim/2/2039-02-'+str(frame)+'.png')

import imageio.v3 as iio

gif_path = "./2039-2_anim.gif"

frames = np.stack([iio.imread(f"./2039/anim/2/2039-02-{x}.png") for x in range(len(nc_data_uu.time))], axis=0)

iio.imwrite(gif_path, frames)